In [1]:
import re

# Part 1

In [2]:
def parseinput(filename,ruleparser):
    register = {}
    pieces = []
    with open(filename) as f:
        for line in f:
            if line=="\n": break
            parseworflow(line.rstrip(),register,ruleparser)
        for line in f:
            pieces.append(parsepiece(line.rstrip()))
    return register,pieces

def parseworflow(wf,register,ruleparser):
    name, rules = re.search("(.*)\{(.*)\}",wf).groups()
    register[name]= [ruleparser(r) for r in rules.split(",")]

def parserulesingle(rule):
    if re.match("^[a-zAR]*$",rule):
        return lambda piece: (True,rule)
    else:
        _, category, lgtr, value, nextworkflow =  re.search("((\w)(<|>)(\d+):(.+))",rule).groups()
        if lgtr =="<":
            return lambda piece : ((piece[category] < int(value)),nextworkflow)
        else:
            return lambda piece : ((piece[category] > int(value)),nextworkflow)


def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return zip(a, a)

def parsepiece(piece):
    xmas =re.search("\{(.)=(\d+),(.)=(\d+),(.)=(\d+),(.)=(\d+)\}",piece).groups()
    return {k:int(v) for k,v in pairwise(xmas)}

def piecerating(piece):
    return sum(piece.values())

In [3]:
register, pieces = parseinput("input19.txt",parserulesingle)

In [4]:
def processpieces(pieces,register):
    ratesum = 0
    for piece in pieces:
        curworkflow = "in"
        while curworkflow in register:
            curworkflow = useworkflow(piece, register[curworkflow])
        if curworkflow=="A" : ratesum += piecerating(piece)
    return ratesum

def useworkflow(piece,workflow):
    for rule in workflow:
        res, todo = rule(piece)
        if res:
            return todo


In [5]:
processpieces(pieces,register)

362930

# Part 2

In [6]:
def parserulesrange(rule):
    if re.match("^[a-zAR]*$",rule):
        return None,rule
    else:
        _, category, lgtr, value, nextworkflow =  re.search("((\w)(<|>)(\d+):(.+))",rule).groups()
        return (category,lgtr,int(value)),nextworkflow

def ratevolume(volume):
    vol = findvolume(volume)
    oneDres = {k:(x[1]+x[0]-1)for k,x in volume.items()}
    rate = 0
    for d in oneDres.values(): rate += (vol*d)//2
    return rate

def findvolume(volume):
    deltas = {k:max((x[1]-x[0]),0) for k,x in volume.items()}
    vol =1
    for v in deltas.values(): vol *=v
    if vol==0: print(volume)
    return vol

def processvolume(volume,workflow,register):
    ratesum = 0
    if workflow=="A": 
        return findvolume(volume)
    elif workflow=="R" or findvolume(volume)==0: return 0

    for rule,destination in register[workflow]:
        if rule is None:
            match destination:
                case "A":
                    ratesum+= findvolume(volume)
                case "R":
                    pass
                case _:
                    ratesum+= processvolume(volume,destination,register)
        else:
            category,lgtr,value = rule
            voldown = volume.copy()
            volup = volume.copy()
            mX,MX = volume[category]
            voldown[category] = (mX,value)
            volup[category] = (value,MX)
            if lgtr =="<":
                if  MX<=value:
                    ratesum += processvolume(volume.copy(),destination,register)
                    break
                elif mX<value:
                    voldown[category] = (mX,value)
                    volup[category] = (value,MX)
                    volume[category] =  volup[category]
                    ratesum += processvolume(voldown,destination,register)
                #Last case is continuing the list of instructions
            else:
                if mX>=value+1:
                    ratesum += processvolume(volume.copy(),destination,register)
                    break
                elif MX>value+1:
                    voldown[category] = (mX,value+1)
                    volup[category] = (value+1,MX)

                    volume[category] =  voldown[category]
                    ratesum += processvolume(volup,destination,register)
                #Last case is continuing the list of instructions
    return ratesum


In [7]:
Allvols =[]
inivol = {k:(1,4001) for k in "xmas"}
registerrange,piecex = parseinput("input19.txt",parserulesrange)
processvolume(inivol,"in",registerrange)

116365820987729

In [8]:
116365820987729/4001**4

0.45409971819933975